Refer here for instructions to [download and run this sample locally](https://developers.arcgis.com/python/sample-notebooks/#Download-and-run-the-sample-notebooks) on your computer

# Creating hurricane tracks using bigdata analytics

The sample code below uses big data analytics (GeoAnalytics) to reconstruct hurricane tracks using data registered on a big data file share in the GIS. Note that this functionality is currently available on ArcGIS Enterprise 10.5 and not yet with ArcGIS Online.

## Reconstruct tracks
Reconstruct tracks is a type of data aggregation tool available under big data tools. This tool works with a layer of point features or polygon features that are time enabled. It first determines which points belong to a track using an identification number or identification string. Using the time at each location, the tracks are ordered sequentially and transformed into a line representing the path of movement.

## Data used
For this sample, hurricane data from over a period of 50 years, totalling about 150,000 points split into 5 shape files was used. The [National Hurricane Center](http://www.nhc.noaa.gov/gis/) provides similar datasets that can be used for exploratory purposes.

To illustrate the nature of the data a subset was published as a feature service and can be visualized as below:

In [1]:
from arcgis import GIS

# Create an anonymous connection to ArcGIS Online
ago_gis = GIS()
subset_search = ago_gis.content.search("Hurricane_tracks_points AND owner:atma.mani", "Feature Service")
subset_search[0]

<Item title:"Hurricane_tracks_points" type:Feature Service owner:atma.mani>

In [2]:
subset_map = ago_gis.map("USA", zoomlevel = 2)
subset_map

In [3]:
subset_map.add_layer(subset_search[0])

## Inspect the data attributes
Let us read the subset as a FeatureLayer to view its attribute table

In [4]:
from pandas.io.json import json_normalize

subset_FL = subset_search[0].layers[0]

query_result = subset_FL.query(where ='FID < 5', 
                                out_fields = "*", 
                                returnGeometry = False)

att_data_frame = json_normalize(query_result)
att_data_frame.columns = att_data_frame.columns.str.replace("attributes.","")
att_data_frame

,ATC_eye,ATC_grade,ATC_poci,ATC_pres,ATC_rmw,ATC_roci,ATC_w34_r1,ATC_w34_r2,ATC_w34_r3,ATC_w34_r4,...,basin_1,day,hour,min_,month,wmo_pres,wmo_pres__,wmo_wind,wmo_wind__,year
0,-999,-999.,-999,-999,-999,-999,-999,-999,-999,-999,...,SI,1,0,0,1,-999,-999,-999,-999,1932
1,-999,-999.,-999,-999,-999,-999,-999,-999,-999,-999,...,SI,1,6,0,1,0,-100,0,-100,1932
2,-999,-999.,-999,-999,-999,-999,-999,-999,-999,-999,...,SI,1,12,0,1,-999,-999,-999,-999,1932
3,-999,-999.,-999,-999,-999,-999,-999,-999,-999,-999,...,SI,1,18,0,1,-999,-999,-999,-999,1932


## Create a data store
For the GeoAnalytics server to process your big data, it needs the data to be registered as a data store. In our case, the data is in multiple shape files and we will register the folder containing the files as a data store of type `bigDataFileShare`.

Let us connect to an ArcGIS Enterprise

In [5]:
gis = GIS("portal or org url","username","password")

The datastore property of GIS provides you with a DatastoreManager object. This object allows you to query, inspect and manipulate the [datastores](http://server.arcgis.com/en/portal/latest/administer/windows/what-is-arcgis-data-store.htm) available to your ArcGIS Server.

In [6]:
# Query the data stores available
data_mgr = gis.datastore
data_mgr.search()

[<Datastore title:"/bigDataFileShares/Fortune_500" type:"bigDataFileShare">,
 <Datastore title:"/bigDataFileShares/NYCdata" type:"bigDataFileShare">,
 <Datastore title:"/enterpriseDatabases/AGSDataStore_ds_t6qywzm8" type:"egdb">,
 <Datastore title:"/fileShares/_raster_store" type:"folder">,
 <Datastore title:"/nosqlDatabases/AGSDataStore_bigdata_bds_jn7cdee2" type:"nosql">,
 <Datastore title:"/nosqlDatabases/AGSDataStore_nosqldb_tcs_5p0kacid" type:"nosql">]

There is no big data file share for hurricane track data registered on the server. So let us register one that points to the shared folder containing the shape files.

In [7]:
data_item = data_mgr.add_bigdata("Hurricane_tracks",
                                 r"\\path_to_hurricane_data")

Created Big Data file share for Hurricane_tracks


Once a big data file share is created, the GeoAnalytics server processes all the valid file types to discern the schema of the data. This process can take a few minutes depending on the size of your data. Once processed, querying the manifest property returns the schema. As you can see from below, the schema is similar to the subset we observed earlier in this sample.

In [8]:
from IPython.display import display
display(data_item.manifest['datasets'][0]) #for brevity only a portion is printed

{'format': {'extension': 'shp', 'type': 'shapefile'},
 'geometry': {'geometryType': 'esriGeometryPoint',
  'spatialReference': {'wkid': 4326}},
 'name': 'h1842_92',
 'schema': {'fields': [{'name': 'Serial_Num', 'type': 'esriFieldTypeString'},
   {'name': 'Season', 'type': 'esriFieldTypeInteger'},
   {'name': 'Num', 'type': 'esriFieldTypeBigInteger'},
   {'name': 'Basin', 'type': 'esriFieldTypeString'},
   {'name': 'Sub_basin', 'type': 'esriFieldTypeString'},
   {'name': 'Name', 'type': 'esriFieldTypeString'},
   {'name': 'ISO_time', 'type': 'esriFieldTypeString'},
   {'name': 'Nature', 'type': 'esriFieldTypeString'},
   {'name': 'wmo_wind', 'type': 'esriFieldTypeSingle'},
   {'name': 'wmo_pres', 'type': 'esriFieldTypeSingle'},
   {'name': 'Center', 'type': 'esriFieldTypeString'},
   {'name': 'wmo_wind__', 'type': 'esriFieldTypeSingle'},
   {'name': 'wmo_pres__', 'type': 'esriFieldTypeSingle'},
   {'name': 'Track_type', 'type': 'esriFieldTypeString'},
   {'name': 'basin_1', 'type': 'esr

## Perform data aggregation using reconstruct tracks tool
When you add a big data file share datastore, a corresponding item gets created on your portal. You can search for it like a regular item and query its layers.

In [9]:
search_result = gis.content.search("", item_type = "big data file share")
search_result

[<Item title:"bigDataFileShares_NYCdata" type:Big Data File Share owner:Admin>,
 <Item title:"bigDataFileShares_Hurricane_tracks" type:Big Data File Share owner:Admin>,
 <Item title:"bigDataFileShares_Fortune_500" type:Big Data File Share owner:Admin>]

In [10]:
data_item = search_result[1]
data_item.url

'https://portalname.domain.com/server/rest/services/DataStoreCatalogs/bigDataFileShares_Hurricane_tracks/BigDataCatalogServer'

In [11]:
years_50 = data_item.layers[0]
years_50

<Layer url:"https://portalname.domain.com/server/rest/services/DataStoreCatalogs/bigDataFileShares_Hurricane_tracks/BigDataCatalogServer/h1842_92">

### Reconstruct tracks tool

The `reconstruct_tracks()` tool can be accessed through the tools.bigdata property of your GIS. In this example, we are using this tool to aggregate the numerous points into line segments showing the tracks followed by the hurricanes. The tool creates a feature service as an output which can be accessed once the processing is complete.

In [12]:
agg_result = gis.tools.bigdata.reconstruct_tracks(years_50,
                                                 track_fields = 'Serial_Num',
                                                 output_name = 'hurricane_tracks_agg_result',
                                                 method = 'GEODESIC')

Submitted.
Executing...
Executing (ReconstructTracks): ReconstructTracks "Feature Set" Serial_Num GEODESIC # # # # {"itemProperties":{"itemId":"0f614d26acb3481fa34f62f1dda0be63"},"serviceProperties":{"serviceUrl":"http://portalname.domain.com/server/rest/services/Hosted/hurricane_tracks_agg_result/FeatureServer","name":"hurricane_tracks_agg_result"}} # # GDB #
Start Time: Thu Sep 15 22:43:45 2016
Using URL based GPRecordSet param: https://portalname.domain.com/server/rest/services/DataStoreCatalogs/bigDataFileShares_Hurricane_tracks/BigDataCatalogServer/h1842_92
Starting new distributed job with 7 tasks.
1/7 distributed tasks completed.
5/7 distributed tasks completed.
7/7 distributed tasks completed.
Finished writing
  extent = Some(Envelope: [-103.0, -38.529998779296875, 109.0999984741211, 55.0])
  interval = Some(Interval(MutableInstant(1842-10-25 06:00:00.000),MutableInstant(1882-11-30 06:00:00.000)))
  count = 618


## Inspect the results
Let us create a map and load the processed result which is a feature service

In [13]:
processed_map = gis.map("USA", zoomlevel = 2)
processed_map

In [14]:
sr = gis.content.search("hurricane_tracks_agg_result")
sr

[<Item title:"hurricane_tracks_agg_result" type:Feature Service owner:Admin>]

In [15]:
processed_map.add_layer(sr[0])

Thus we transformed a bunch of ponints into tracks that represents paths taken by the hurricanes over a period of 50 years. We can pull up another map and inspect the results a bit more closely

In [16]:
map2 = gis.map('USA', zoomlevel = 2)
map2

In [17]:
tracks_layer = sr[0].layers[0]
tracks_layer

<FeatureLayer url:"http://portalname.domain.com/server/rest/services/Hosted/hurricane_tracks_agg_result/FeatureServer/0">

In [18]:
map2.add_layer(tracks_layer)

Our input data and the map widget is time enabled. Thus we can filter the data to represent the tracks from only the years 1852 to 1860

In [19]:
map2.start_time = '1852'
map2.end_time = '1860'

## What can big data tools do for you?

With this sample we just scratched the surface of what big data analysis can do for you. ArcGIS Enterprise at 10.5 packs a powerful set of tools that let you derive a lot of value from your data. You can do so by asking the right questions, for instance, a weather dataset such as this could be used to answer a few interesting questions such as
 
 - did the number of hurricanes per season increase over the years?
 - give me the hurricanes that travelled longest distance
 - give me the ones that stayed for longest time. Do we see a trend?
 - how are wind speed and distance travelled correlated?
 - my assets are located in a tornado corridor. How many times in the past century, was there a hurricane within 50 miles from my assets?
 - my industry is dependent on tourism, which is heavily impacted by the vagaries of weather. From historical weather data, can I correlate my profits with major weather events? How well is my business insulated from freak weather events?
 - over the years do we see any shifts in major weather events - do we notice a shift in when the hurricane season starts?
 
The ArcGIS Python API gives you a gateway to easily access the big data tools from your ArcGIS Enterprise. By combining it with other powerful libraries from the pandas and scipy stack and the rich visualization capabilities of the Jupyter notebook, you can extract a lot of value from your data, big or small.